In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style="whitegrid")

data = pd.read_csv('schedule_v003.csv', low_memory=False)

# Convert booking from string into a datatime.
data['Booking'] = pd.to_datetime(data['Booking'], format='%Y-%m-%d %H:%M:%S')

# This is used later on. 
day_names = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Convert time columns into their correct datatypes.
for column in ['Arrived', 'Started', 'Completed']:
    data[column] = pd.to_datetime(data[column], format='%H:%M:%S').dt.time

data.head()

In [2]:
# Show how frequently each location is used
keys = data['LocationName'].value_counts().keys().tolist()
values = data['LocationName'].value_counts().tolist()
counts = pd.DataFrame([(row[0].strip(), row[1]) for row in zip(keys, values)])
counts.columns=['LocationName', 'Count']

fig, ax = plt.subplots(figsize=(30, 20))
sns.barplot(ax=ax, x='LocationName', y='Count', data=counts).set_title('Location Counts')
fig.savefig('charts/location_counts.png')

In [3]:
# Show how frequent each activity is
keys = data['Activity'].value_counts().keys().tolist()
values = data['Activity'].value_counts().tolist()
counts = pd.DataFrame([(row[0].strip(), row[1]) for row in zip(keys, values)])
counts.columns=['Activity', 'Count']

fig, ax = plt.subplots(figsize=(100, 20))
sns.barplot(ax=ax, x='Activity', y='Count', data=counts).set_title('Activity Counts')
fig.savefig('charts/activity_counts.png')
counts.describe()

In [4]:
keys = data['DepartmentName'].value_counts().keys().tolist()
values = data['DepartmentName'].value_counts().tolist()
counts = pd.DataFrame([(row[0].strip(), row[1]) for row in zip(keys, values)])
counts.columns=['DepartmentName', 'Count']

fig, ax = plt.subplots(figsize=(15, 15))
sns.barplot(x='DepartmentName', y='Count', data=counts).set_title('Department Counts')
fig.savefig('charts/department_counts.png')
counts.describe()

In [5]:
import sqlite3

#Set up the database
db = sqlite3.connect('sql/gcpace.sqlite3')
cur = db.cursor()

In [6]:
cur.execute("SELECT ROUND(waiting_time / 5) * 5 AS C FROM bookings WHERE waiting_time > 0 AND started > arrived")
frame = pd.DataFrame([int(x[0]) for x in cur.fetchall()])
frame.columns = ['WaitingTimes']

keys = frame['WaitingTimes'].value_counts().keys().tolist()
values = frame['WaitingTimes'].value_counts().tolist()
counts = pd.DataFrame([(row[0], row[1]) for row in zip(keys, values)])
counts.columns=['WaitingTimes', 'Count']

# There are a number of extreme outliers in this DataFrame. Remove all points that are not within 3 standard deviations
quantile = counts["Count"].quantile(0.997)
counts = counts[counts["Count"] < quantile]

fig, ax = plt.subplots(figsize=(20, 15))
sns.barplot(x='WaitingTimes', y='Count', data=counts).set_title('Waiting Time Counts')
fig.savefig('charts/waiting_time_counts.png')

In [7]:
cur.execute("SELECT ROUND(treatment_time / 5) * 5 AS C FROM bookings WHERE treatment_time > 0 AND completed > started")
frame = pd.DataFrame([int(x[0]) for x in cur.fetchall()])
frame.columns = ['TreatmentTimes']

keys = frame['TreatmentTimes'].value_counts().keys().tolist()
values = frame['TreatmentTimes'].value_counts().tolist()
counts = pd.DataFrame([(row[0], row[1]) for row in zip(keys, values)])
counts.columns=['TreatmentTimes', 'Count']

# There are a number of extreme outliers in this DataFrame. Remove all points that are not within 3 standard deviations
quantile = counts["Count"].quantile(0.997)
counts = counts[counts["Count"] < quantile]

fig, ax = plt.subplots(figsize=(20, 15))
sns.barplot(x='TreatmentTimes', y='Count', data=counts).set_title('Treatment Time Counts')
fig.savefig('charts/treatment_time_counts.png')

In [10]:
cur.execute("SELECT id, code FROM activities ORDER BY id ASC")
activities = [(x[0], x[1].strip()) for x in cur.fetchall()]

data = []
for activity in activities:
    cur.execute(
        "SELECT ROUND(treatment_time / 5) * 5 AS C FROM bookings "
        "WHERE treatment_time > 0 AND completed > started and activity_id=?", [activity[0]]
    )
    
    for row in cur.fetchall():
        data.append([activity[1], int(row[0])])
    
frame = pd.DataFrame(data, columns=['activity', 'time'])

# There are a number of extreme outliers in this DataFrame. Remove all points that are not within 3 standard deviations
quantile = frame["time"].quantile(0.997)
frame = frame[frame["time"] < quantile]

fig, ax = plt.subplots(figsize=(30, 20))
sns.boxplot(ax=ax, x='time', y='activity', data=frame).set_title('Treatment Times By Activity')
fig.savefig('charts/activity_treatment_counts.png')

In [11]:
# Get all bookings and arrange by day of the week & booking time (hours).

cur.execute("""
    SELECT locations.name, cast (strftime('%w', booking_datetime) as integer) AS DWEEK, COUNT(bookings.schedule_id) AS C FROM bookings 
    INNER JOIN locations ON locations.id = bookings.location_id 
    GROUP BY locations.name, DWEEK
    ORDER BY name, DWEEK
""")

rows = [(x[0].strip(), day_names[x[1]], x[2]) for x in cur.fetchall()]
frame = pd.DataFrame(rows)
frame.columns = ['MachineName', 'DayOfWeek', 'Count']

pivot = frame.pivot(index='MachineName', columns='DayOfWeek', values='Count')
# Reindex to order column names in correct ordering.
pivot = pivot.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'], axis=1)

fig, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(pivot, ax=ax).set_title('Machine Utilisation By Day')
fig.savefig('charts/machine_utilisation_heatmap.png')

In [12]:
# Get all bookings and arrange by day of the week & booking time (hours).

cur.execute("""
    SELECT cast (strftime('%w', booking_datetime) as integer) AS DID, strftime('%H', booking_datetime) AS DHOUR, COUNT(*) FROM bookings
    GROUP BY DID, DHOUR
    ORDER BY DID, DHOUR ASC
""")

rows = [(day_names[x[0]], x[1], x[2]) for x in cur.fetchall()]
frame = pd.DataFrame(rows)
frame.columns = ['DayName', 'Hour', 'Count']

order = ['Monday', 'Tuesday', 'Wednesday','Thursday','Friday','Saturday','Sunday']
pivot = frame.pivot(index='DayName', columns='Hour', values='Count').reindex(order)
fig, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(pivot, ax=ax).set_title('Bookings Per Hour')
fig.savefig('charts/booking_heatmap.png')

In [13]:
cur.execute("""
    SELECT cast (strftime('%w', booking_datetime) as integer) AS DOW, ROUND(delay_time / 5) * 5 AS DTIME, COUNT(*) FROM bookings
    GROUP BY DOW, DTIME
    ORDER BY DOW ASC, DTIME DESC
""")

rows = [(day_names[x[0]], x[1], x[2]) for x in cur.fetchall()]

frame = pd.DataFrame(rows)
frame.columns = ['DayName', 'DelayTime', 'Count']
    
# There are a number of extreme outliers in this DataFrame. Remove all points that are not within 3 standard deviations
quantile = frame["Count"].quantile(0.997)
frame = frame[frame["Count"] < quantile]

for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:
    fig, ax = plt.subplots(figsize=(75, 15))
    sns.barplot(x='DelayTime', y='Count', data=frame.loc[frame['DayName'] == day]).set_title('Delay Time For ' + day)
    fig.savefig('charts/{}_delay_time.png'.format(day))
    plt.close(fig)

In [14]:
cur.execute("""
    SELECT patient_id, count(*) FROM bookings
    GROUP BY patient_id
""")

rows = [x[1] for x in cur.fetchall()]

frame = pd.DataFrame(rows)
frame.columns = ['Count']

fig, ax = plt.subplots(figsize=(10, 10))
sns.boxplot(x='Count', data=frame).set_title('Patient Treatment Counts')
fig.savefig('charts/patient_treatment_count.png')

In [15]:
# Calculate how the percent change between each treatment times group by patient.
cur.execute("""SELECT patient_id, treatment_time, booking_datetime, code FROM bookings
    INNER JOIN activities ON activity_id = activities.id
    WHERE treatment_time > 0
    ORDER BY patient_id, booking_datetime""")

current_patient = None
first_treatment = None
num_treatment = None

data = []
for row in cur.fetchall():
    if current_patient != row[0]:
        current_patient = row[0]
        first_treatment = row[1]
        current_treatment = 1
        continue
    
    difference = row[1] - first_treatment
    percent = (difference / first_treatment) * 100
    data.append([row[3], current_treatment, percent])
    current_treatment += 1

frame = pd.DataFrame(data, columns=['Activity', 'TreatmentNumber', 'TimeDifference'])

# There are a number of extreme outliers in this DataFrame. Remove all points that are not within 3 standard deviations
quantile = frame["TimeDifference"].quantile(0.997)
frame = frame[frame["TimeDifference"] < quantile]

for activity in frame['Activity'].unique():
    fig, ax = plt.subplots(figsize=(45, 45))
    sns.stripplot(x='TreatmentNumber', y='TimeDifference', data=frame[frame['Activity'] == activity], dodge=False, jitter=False, palette='Set2')
    fig.savefig('charts/treatment_time_differences/{}.png'.format(activity))
    plt.close(fig)

In [16]:

from datetime import datetime
from math import floor

def getTimeDifference(dt1, dt2):
    if dt2 < dt1:
        return -getTimeDifference(dt2, dt1)

    delta = dt2 - dt1
    # Round down and remove any decimal places. We only care about whole minutes
    return int(floor(delta.seconds / 60))

cur.execute("""
SELECT strftime('%Y-%m-%d', booking_datetime), location_id, department_id, started, completed, activity_id, locations.name FROM bookings
INNER JOIN locations ON location_id = locations.id
ORDER BY department_id, location_id
""")

rows = cur.fetchall()
previous = rows[0]

results = []
for row in rows[1:]:
    # Skip to the next entry on change of day or change of location
    if row[1] != previous[1] or row[0] != previous[0]:
        previous = row
        continue
    
    completed = datetime.strptime(previous[4], '%H:%M:%S')
    start = datetime.strptime(row[3], '%H:%M:%S')
    
    minutes = getTimeDifference(start, completed)
    results.append(tuple([row[6], minutes, row[5] == previous[5]]))
    previous = row
    
frame = pd.DataFrame(results, columns=['Location', 'Delta', 'SameActivity'])

# There are a number of extreme outliers in this DataFrame. Remove all points that are not within 3 standard deviations
quantile = frame["Delta"].quantile(0.997)
frame = frame[frame["Delta"] < quantile]

fig, ax = plt.subplots(figsize=(30, 30))
sns.stripplot(x='Location', y='Delta', hue='SameActivity', 
              data=frame, jitter=True, palette='Set2', dodge=True)

fig.savefig('charts/machine_booking_between_time.png')

In [29]:
cur.execute("""
SELECT bookings.treatment_time, prediction, activities.code FROM bookings
INNER JOIN activities ON activity_id = activities.id
INNER JOIN booking_gc_predict ON bookings.schedule_id = booking_gc_predict.schedule_id
WHERE bookings.treatment_time > 0
""")

result = [((x[0] - x[1]), x[2]) for x in cur.fetchall()]
df = pd.DataFrame(result, columns=['Difference', 'Activity'])

fig, ax = plt.subplots(figsize=(100, 30))
sns.boxenplot(x='Activity', y='Difference', data=df, ax=ax)
fig.savefig('charts/gc_predictions_vs_reality.png')